<p align="center">
  <span style="font-size:18px"><b>ISEL - DEETC</b></span><br><br>
  <span style="font-size:22px"><b>Processamento Digital de Sinais</b></span><br><br>
  <span style="font-size:18px"><b>1º Trabalho Prático</b></span><br>
  <span style="font-size:16px">2º Semestre 2024/2025</span><br><br>
  <span style="font-size:16px">Docente: Prof.</span><br>
  <span style="font-size:16px">Data: 21/5/2025</span><br><br>
  <span style="font-size:16px">Trabalho realizado por:</span><br>
  <span style="font-size:16px">Artur Assis nº52553</span><br>
  <span style="font-size:16px">David Santos nº51417</span><br>
  <span style="font-size:16px">Bernardo Aguiar nº52483</span>
</p>


### 5.
O processamento de informação geográfica é uma ferramenta omnipresente no nosso dia a dia. Pretende-se
estudar treinos associados a vários desportos. Foi criado um dataset associados a vários utilizadores da aplicação
EndoMondo. Serão disponibilizado vários ficheiros json onde será possível explorar a dinâmica dos vários treinos.
Pretende-se que sejam explorados usando a biblioteca ipyleaflet. Pretende-se que seja desenhadas as etapas,
calculados o nº de km a subir e a descer, determinada a velocidade média (em todo o percurso e separadamente
nos troços a subir e a descer), a distância em plano, etc. Devem comparar a evolução do ritmo cardíaco em
vários percursos e relaciona-lo com as zonas de treino.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipyleaflet import Map, basemaps, GeoJSON
from geopy.distance import geodesic
import json


def create_map(longitudes, latitudes):
    coordinates = list(zip(latitudes, longitudes))
    start_line = coordinates[0]
    m = Map(center=start_line, zoom=10, basemap=basemaps.OpenStreetMap.HOT)
    geojson = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [[lon, lat] for lat, lon in coordinates],
        },
    }
    line = GeoJSON(data=geojson, style={"color": 'red', "weight": 2})
    m.add_layer(line)
    display(m)
    
def draw_heart_rate(heart_rates, timestamps):
    time_minutes = [(t - timestamps[0]) / 60 for t in timestamps]
    plt.figure(figsize=(12, 4))
    plt.plot(time_minutes, heart_rates, label='Frequência Cardíaca', color='blue')
    plt.title('Frequência Cardíaca ao Longo do Tempo')
    plt.xlabel('Tempo (minutos)')
    plt.ylabel('Frequência Cardíaca (bpm)')
    plt.xlim(0, (timestamps[-1] - timestamps[0]) / 60)
    plt.grid()
    plt.legend()
    plt.show()
    
def calculate_lthr(timestamps, heart_rates):
    for current_timestamp in timestamps:
        if current_timestamp - timestamps[0] >= 10 * 60:
            starting_timestamp = current_timestamp
            break
        
    for current_timestamp in timestamps:
        if current_timestamp - timestamps[0] >= 30 * 60:
            final_timestamp = current_timestamp
            break

    hr_last_20min = [hr for t, hr in zip(timestamps, heart_rates) if t >= starting_timestamp and t <= final_timestamp]

    lthr = sum(hr_last_20min) / len(hr_last_20min)
    return lthr
        
def draw_zones_over_time(timestamps, heart_rates, lthr, sport_type):
    zonas = [classify_zone(hr, lthr, sport_type) for hr in heart_rates]
    zonas_labels = ['Zona 1', 'Zona 2', 'Zona 3', 'Zona 4', 'Zona 5a', 'Zona 5b', 'Zona 5c']
    zonas_numeros = [zonas_labels.index(z) + 1 if z in zonas_labels else 0 for z in zonas]
    tempo_min = [(t - timestamps[0]) / 60 for t in timestamps]

    plt.figure(figsize=(12, 4))
    plt.step(tempo_min, zonas_numeros, where='post', color='purple')
    plt.yticks(range(1, 8), zonas_labels)
    plt.xlabel('Tempo (minutos)')
    plt.ylabel('Zona de Treino')
    plt.title('Zona de Treino ao Longo do Tempo')
    plt.grid(axis='y')
    plt.tight_layout()
    plt.xlim(0, (timestamps[-1] - timestamps[0]) / 60)
    plt.ylim(0, 8)
    plt.show()
    
def draw_altitude_over_time(timestamps, altitudes):
    time_minutes = [(t - timestamps[0]) / 60 for t in timestamps]
    plt.figure(figsize=(12, 4))
    plt.plot(time_minutes, altitudes, label='Altitude', color='green')
    plt.title('Altitude ao Longo do Tempo')
    plt.xlabel('Tempo (minutos)')
    plt.ylabel('Altitude (m)')
    plt.xlim(0, (timestamps[-1] - timestamps[0]) / 60)
    plt.grid()
    plt.legend()
    plt.show()

def draw_speed_over_time(timestamps, speeds):
    time_minutes = [(t - timestamps[0]) / 60 for t in timestamps]
    plt.figure(figsize=(12, 4))
    plt.plot(time_minutes, speeds, label='Velocidade', color='orange')
    plt.title('Velocidade ao Longo do Tempo')
    plt.xlabel('Tempo (minutos)')
    plt.ylabel('Velocidade (m/s)')
    plt.xlim(0, (timestamps[-1] - timestamps[0]) / 60)
    plt.grid()
    plt.legend()
    plt.show()
    
def classify_zone(heart_rate, lthr, sport_type):
    if sport_type == 'run':
        if heart_rate < 0.85 * lthr:
            return 'Zona 1'
        elif heart_rate <= 0.89 * lthr:
            return 'Zona 2'
        elif heart_rate <= 0.94 * lthr:
            return 'Zona 3'
        elif heart_rate <= 0.99 * lthr:
            return 'Zona 4'
        elif heart_rate <= 1.02 * lthr:
            return 'Zona 5a'
        elif heart_rate <= 1.06 * lthr:
            return 'Zona 5b'
        elif heart_rate > 1.06 * lthr:
            return 'Zona 5c'
    elif sport_type == 'bike':
        if heart_rate < 0.81 * lthr:
            return 'Zona 1'
        elif heart_rate <= 0.89 * lthr:
            return 'Zona 2'
        elif heart_rate <= 0.93 * lthr:
            return 'Zona 3'
        elif heart_rate <= 0.99 * lthr:
            return 'Zona 4'
        elif heart_rate <= 1.02 * lthr:
            return 'Zona 5a'
        elif heart_rate <= 1.06 * lthr:
            return 'Zona 5b'
        elif heart_rate > 1.06 * lthr:
            return 'Zona 5c'
    
def calculate_distance_and_time(latitudes, longitudes, altitudes, timestamps):
    ascent, descent = 0, 0
    ascent_distance, descent_distance, flat_distance = 0, 0, 0
    ascent_time, descent_time, flat_time = 0, 0, 0
    total_distance = 0
    total_time = 0

    for i in range(1, len(latitudes)):
        altitude_difference = altitudes[i] - altitudes[i - 1]
        time_difference = timestamps[i] - timestamps[i - 1]
        current_distance = calculate_total_distance((latitudes[i - 1], longitudes[i - 1]), (latitudes[i], longitudes[i]))
        total_distance += current_distance
        total_time += time_difference
        if altitude_difference > 0:
            ascent += altitude_difference
            ascent_distance += current_distance
            ascent_time += time_difference
        elif altitude_difference < 0:
            descent += abs(altitude_difference)
            descent_distance += current_distance
            descent_time += time_difference
        else:
            flat_distance += current_distance
            flat_time += time_difference

    hours = int(total_time // 3600) 
    minutes = int((total_time % 3600) // 60)
    seconds = int(total_time % 60)
    formatted_time = f"{hours:02}:{minutes:02}:{seconds:02}"
    ascent_speed = (ascent_distance/1000) / (ascent_time / 3600) if ascent_time > 0 else 0
    descent_speed = (descent_distance/1000) / (descent_time / 3600) if descent_time > 0 else 0
    flat_speed = (flat_distance/1000) / (flat_time / 3600) if flat_time > 0 else 0
    
    print(f"Tempo total do percurso: {formatted_time}")
    print(f"Subida acumulada: {ascent:.2f} m")
    print(f"Descida acumulada: {descent:.2f} m")
    print(f"Distância percorrida a subir: {ascent_distance:.2f} m")
    print(f"Distância percorrida a descer: {descent_distance:.2f} m")
    print(f"Distância percorrida em plano: {flat_distance:.2f} m")
    print(f"Velocidade média a subir: {ascent_speed:.2f} km/h")
    print(f"Velocidade média a descer: {descent_speed:.2f} km/h")
    print(f"Velocidade média em plano: {flat_speed:.2f} km/h")
    hours = total_time / 3600.0
    km_distance = total_distance / 1000.0
    average_total_speed = km_distance / hours if hours > 0 else 0
    print(f"Velocidade média global: {average_total_speed:.2f} km/h")
    print(f"Distância total percorrida: {total_distance:.2f} m")
    
def calculate_total_distance(p1, p2):
    return geodesic(p1, p2).meters
        
def all_workouts_distance():
    total_distance = 0
    for file_number in range(50):
        json_filename = f'DatasetEndoMondo/endomondoHR_proper_subset{file_number}.json'
        try:
            with open(json_filename, 'r', encoding='utf-8') as f:
                file_data = json.load(f)
                for workout in file_data.values():
                        longitudes = workout.get('longitude', [])
                        latitudes = workout.get('latitude', [])
                        altitudes = workout.get('altitude', [])
                        for i in range(1, len(latitudes)):  
                            total_distance += np.sqrt(
                                (latitudes[i] - latitudes[i - 1]) ** 2 +
                                (longitudes[i] - longitudes[i - 1]) ** 2 +
                                (altitudes[i] - altitudes[i - 1]) ** 2
                            )
        except FileNotFoundError:
            print(f"O ficheiro {json_filename} não foi encontrado.")
        except json.JSONDecodeError:
            print(f"Erro ao decodificar JSON no ficheiro {json_filename}.")
    
    print(f"\nDistância total de todos os treinos em metros: {total_distance:.2f} m")
    print(f"\nDistância total de todos os treinos em kilómetros: {total_distance / 1000:.2f} km")



# INICIO DO MAIN 
  
workouts_data = {}
workout_id = '392337038' # ESCOLHER MANUALMENTE O ID DO TREINO A EXIBIR
for file_number in range(50):
    json_filename = f'DatasetEndoMondo/endomondoHR_proper_subset{file_number}.json'
    try:
        with open(json_filename, 'r', encoding='utf-8') as f:
            file_data = json.load(f)
            workouts_data.update(file_data)
    except FileNotFoundError:
        print(f"O ficheiro {json_filename} não foi encontrado.")
    except json.JSONDecodeError:
        print(f"Erro ao decodificar JSON no ficheiro {json_filename}.")
    
workout_ids = list(workouts_data.keys())

if workout_ids:
    if workout_id in workouts_data:
        workout = workouts_data[workout_id]
        
        longitudes = workout.get('longitude', [])
        latitudes = workout.get('latitude', [])
        altitudes = workout.get('altitude', [])
        heart_rates = workout.get('heart_rate', [])
        timestamps = workout.get('timestamp', [])
        speeds = workout.get('speed', [])

        print(f"\n--- Dados do treino (ID: {workout_id}) ---")
        print(f"Desporto: {workout.get('sport', 'Não especificado')}")
        print(f"ID d0 utilizador: {workout.get('userId', 'Não especificado')}")
        print(f"Género do utilizador: {workout.get('gender', 'Não especificado')}")

        create_map(longitudes, latitudes)
        draw_heart_rate(heart_rates, timestamps)
        
        lthr = calculate_lthr(timestamps, heart_rates)
        draw_zones_over_time(timestamps, heart_rates, lthr, workout.get('sport', 'run'))
        draw_altitude_over_time(timestamps, altitudes)
        draw_speed_over_time(timestamps, speeds)
        
        print("===== Resumo do Treino =====")
        calculate_distance_and_time(latitudes, longitudes, altitudes, timestamps)
        print(f"LTHR estimado: {lthr:.0f} bpm")
        print("============================\n")
        all_workouts_distance()

else:
    print("\nO dicionário de treinos está vazio.")